In [2]:
from datasets import load_dataset, logging
logging.set_verbosity_debug()
accent_coach_dataset = load_dataset("preetam8/accent_coach_training_dataset")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Overwrite dataset info from restored data version if exists.
Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
Found cached dataset accent_coach_training_dataset (/Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee)
Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
Constructing Dataset for split train, validation, test, from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee


Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

In [3]:
import numpy as np

from datasets import DatasetDict, Sequence, Value, Features
from transformers import SpeechT5Processor

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")

def preprocess_dataset(source_audio, target_audio):
    processor_output = processor(
        audio=source_audio["array"],
        audio_target=target_audio["array"],
        sampling_rate=16_000,
        return_attention_mask=False,
    )
    input_values = np.array(processor_output.input_values[0], dtype=np.float16)
    labels = np.array(processor_output.labels[0], dtype=np.float32)
    return {"input_values": input_values, "labels": labels}

final_dataset = {}
features = Features({
    "input_values": Sequence(Value(dtype="float16")),
    "labels": Sequence(Sequence(Value(dtype="float32"))),
})
for split in accent_coach_dataset:
    final_dataset[split] = accent_coach_dataset[split].map(
        preprocess_dataset,
        input_columns=["source_audio", "target_audio"],
        features=features,
        remove_columns=accent_coach_dataset[split].column_names
    )

final_dataset = DatasetDict(final_dataset)

Loading cached processed dataset at /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee/cache-b111efdbedde0a4e.arrow
Loading cached processed dataset at /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee/cache-b65915d19f85f44f.arrow
Loading cached processed dataset at /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee/cache-743234b805f909c0.arrow


In [4]:
train_dataset = final_dataset["train"].with_format("torch")
validation_dataset = final_dataset["validation"].with_format("torch")
test_dataset = final_dataset["test"].with_format("torch")
small_validation_dataset = validation_dataset.select(range(10))

Set __getitem__(key) output type to torch for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for no columns  (when key is int or slice) and don't output other (un-formatted) columns.
Set __getitem__(key) output type to torch for no columns  (when key is int or slice) and don't output other (un-formatted) columns.


In [4]:
import logging
import torch
from custom_speecht5_vc import SpeechT5ForSpeechToSpeechForTraining

logging.getLogger().setLevel(logging.INFO)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
INPUT_DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
OUTPUT_DTYPE = torch.float32
ATTENTION_DTYPE = torch.long

logging.info(f"Using device: {DEVICE}, Input dtype: {INPUT_DTYPE}, Output dtype: {OUTPUT_DTYPE}")
model = SpeechT5ForSpeechToSpeechForTraining.from_pretrained("microsoft/speecht5_vc")
model = model.to(device=DEVICE)
model = model.train()

INFO:root:Using device: cpu, Input dtype: torch.float32, Output dtype: torch.float32
Some weights of SpeechT5ForSpeechToSpeechForTraining were not initialized from the model checkpoint at microsoft/speecht5_vc and are newly initialized: ['speecht5.encoder.prenet.pos_sinusoidal_embed.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# from torch.nn.utils.rnn import pad_sequence
# def create_batch(features):
#     input_values = [torch.Tensor(f["input_values"]) for f in features]
#     labels = [torch.Tensor(f["labels"]) for f in features]

#     input_values = pad_sequence(input_values, batch_first=True, padding_value=-100).to(dtype=INPUT_DTYPE, device=DEVICE)
#     labels = pad_sequence(labels, batch_first=True, padding_value=-100).to(dtype=OUTPUT_DTYPE, device=DEVICE)
#     if labels.shape[1] % 2 == 1:
#         pad = torch.ones((labels.shape[0], 1, labels.shape[2])).to(dtype=OUTPUT_DTYPE, device=DEVICE)
#         labels = torch.cat((labels, pad), dim=1)
#     attention_mask = (input_values != -100).to(dtype=ATTENTION_DTYPE, device=DEVICE)
#     decoder_attention_mask = (labels != -100).to(dtype=ATTENTION_DTYPE, device=DEVICE)
    

#     return {
#         "input_values": input_values,
#         "labels": labels,
#         "attention_mask": attention_mask,
#         "decoder_attention_mask": decoder_attention_mask,
#     }

# small_train_dataset = train_dataset.select(range(64))
# model_inputs = create_batch(small_train_dataset)
# with torch.profiler.profile(
#     activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
#     on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
# ) as prof:
#     outputs = model(**model_inputs)
# print(prof.key_averages().table(sort_by="cuda_time_total"))

In [6]:
import torch
from torch.nn.utils.rnn import pad_sequence

class DataCollatorWithPadding:

    def __call__(self, features):
        # tensor_conversion_start = time.time()
        input_values = [f["input_values"] for f in features]
        labels = [f["labels"] for f in features]
        # tensor_conversion_end = time.time()
        # tensor_conversion_total = tensor_conversion_end - tensor_conversion_start

        # padding_start = time.time()
        input_values = pad_sequence(input_values, batch_first=True, padding_value=-100).to(dtype=INPUT_DTYPE, device=DEVICE)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100).to(dtype=OUTPUT_DTYPE, device=DEVICE)
        # padding_end = time.time()
        # padding_total = padding_end - padding_start

        # extra_padding_start = time.time()
        if labels.shape[1] % 2 == 1:
            pad = (torch.ones((labels.shape[0], 1, labels.shape[2]))*-100).to(dtype=OUTPUT_DTYPE, device=DEVICE)
            labels = torch.cat((labels, pad), dim=1)
        # extra_padding_end = time.time()
        # extra_padding_total = extra_padding_end - extra_padding_start

        # am_start = time.time()
        attention_mask = (input_values != -100).to(dtype=ATTENTION_DTYPE, device=DEVICE)
        decoder_attention_mask = (labels != -100).to(dtype=ATTENTION_DTYPE, device=DEVICE)
        # am_end = time.time()
        # am_total = am_end - am_start

        return {
            "input_values": input_values,
            "labels":labels,
            "attention_mask": attention_mask,
            "decoder_attention_mask": decoder_attention_mask,
        }#, padding_total, extra_padding_total, am_total, tensor_conversion_total
    
data_collator = DataCollatorWithPadding()

In [7]:
from transformers import Trainer, TrainingArguments

HUGGINGFACE_TOKEN = "" # To be set/loaded securely
USE_FP16 = True if DEVICE == "cuda" else False
USE_CPU = True if DEVICE == "cpu" else False

TRAINING_ARGUMENTS = {
    "default": {
        "output_dir": "./speecht5_vc_finetuned_accent_coach",
        "eval_strategy": "steps",
        "save_strategy": "steps",
        "learning_rate": 1e-5,
        "weight_decay": 0.01,
        "logging_dir": "./logs",
        "report_to": "tensorboard",
    },
    "cuda": {
        "per_device_train_batch_size": 32,
        "per_device_eval_batch_size": 32,
        "num_train_epochs": 10,
        "logging_steps": 10,
        "save_steps": 10,
        "eval_steps": 10,
        "fp16": True,
        "use_cpu": False,
        "push_to_hub": True,
        "hub_model_id": "preetam8/speecht5_vc_finetuned_accent_coach",
        "hub_strategy": "checkpoint",
        "hub_token": HUGGINGFACE_TOKEN,
        "save_total_limit": 1,
        "dataloader_pin_memory": False,
        "label_names": ["labels"],
        "load_best_model_at_end": True,
        "metric_for_best_model": "loss",
        "greater_is_better": False,
        "eval_on_start": True,
    },
    "cpu": {
        "per_device_train_batch_size": 8,
        "per_device_eval_batch_size": 8,
        "num_train_epochs": 1,
        "logging_steps": 2,
        "save_steps": 100,
        "eval_steps": 100,
        "fp16": False,
        "use_cpu": True,
        "label_names": ["labels"],
    }
}

device_args = TRAINING_ARGUMENTS["cuda"] if DEVICE == "cuda" else TRAINING_ARGUMENTS["cpu"]
training_args = TrainingArguments(**TRAINING_ARGUMENTS["default"], **device_args)

# training_args = TrainingArguments(
#     output_dir="./speecht5",
#     eval_strategy="steps",
#     save_strategy="steps",
#     learning_rate=1e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=1,
#     weight_decay=0.01,logging_dir="./logs",
#     logging_steps=75,
#     save_steps=225,
#     eval_steps=225,
#     report_to="none",
#     fp16=USE_FP16,
#     use_cpu=USE_CPU,
# )

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=small_validation_dataset,
    data_collator=data_collator,
)

In [8]:
trainer.evaluate()

True False False


  0%|          | 0/2 [00:00<?, ?it/s]

True False False


{'eval_loss': 2.560163736343384,
 'eval_model_preparation_time': 0.0047,
 'eval_runtime': 3.5594,
 'eval_samples_per_second': 2.809,
 'eval_steps_per_second': 0.562}

# Trained Model

In [1]:
from custom_speecht5_vc import SpeechT5ForSpeechToSpeechForTraining
from transformers import SpeechT5ForSpeechToSpeech, SpeechT5HifiGan, SpeechT5Processor

fine_tuned_model = SpeechT5ForSpeechToSpeechForTraining.from_pretrained("preetam8/speecht5_vc_finetuned_accent_coach")
pre_trained_model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")


Some weights of SpeechT5ForSpeechToSpeech were not initialized from the model checkpoint at microsoft/speecht5_vc and are newly initialized: ['speecht5.encoder.prenet.pos_sinusoidal_embed.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
from datasets import load_dataset, logging
logging.set_verbosity_debug()
test_dataset = load_dataset("preetam8/accent_coach_training_dataset", split="test")
sample_idx = 15
example_speech = test_dataset[sample_idx]["source_audio"]["array"]
print(test_dataset[sample_idx]["sentence"])
inputs = processor(audio=example_speech, sampling_rate=16_000, return_tensors="pt")

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Overwrite dataset info from restored data version if exists.
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
INFO:datasets.info:Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
Found cached dataset accent_coach_training_dataset (/Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee)
INFO:datasets.builder:Found cached dataset accent_coach_training_dataset (/Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee)
Loading Dataset info from /Users/preetamamancharla/.cache/hug

Antsirabe est le siège d'un évêché catholique.


In [3]:
pre_trained_generated_speech = pre_trained_model.generate_speech(inputs["input_values"], vocoder=vocoder)

prob: tensor([[9.4942e-10, 3.0860e-05]]), tensor([3.0861e-05])
prob: tensor([[6.2035e-10, 4.6443e-06]]), tensor([4.6449e-06])
prob: tensor([[3.9516e-09, 1.1350e-06]]), tensor([1.1389e-06])
prob: tensor([[1.3715e-09, 7.3477e-08]]), tensor([7.4848e-08])
prob: tensor([[7.7674e-10, 5.8665e-08]]), tensor([5.9442e-08])
prob: tensor([[5.6206e-10, 1.4671e-07]]), tensor([1.4727e-07])
prob: tensor([[2.7906e-10, 3.6169e-07]]), tensor([3.6197e-07])
prob: tensor([[3.7564e-10, 9.5560e-08]]), tensor([9.5936e-08])
prob: tensor([[2.5022e-10, 2.4891e-07]]), tensor([2.4916e-07])
prob: tensor([[2.5768e-09, 1.2865e-07]]), tensor([1.3123e-07])
prob: tensor([[4.1959e-09, 2.2405e-06]]), tensor([2.2447e-06])
prob: tensor([[1.7323e-09, 1.1083e-06]]), tensor([1.1100e-06])
prob: tensor([[1.4638e-09, 4.5565e-07]]), tensor([4.5711e-07])
prob: tensor([[3.7621e-10, 1.9582e-07]]), tensor([1.9620e-07])
prob: tensor([[4.3792e-10, 4.0124e-07]]), tensor([4.0168e-07])
prob: tensor([[8.5048e-10, 1.2478e-07]]), tensor([1.256

In [4]:
fine_tuned_generated_speech = fine_tuned_model.generate_speech(inputs["input_values"], vocoder=vocoder, maxlenratio=2.0)

prob: tensor([[2.0979e-07, 3.5956e-05]]), tensor([3.6165e-05])
prob: tensor([[4.6300e-05, 1.5675e-06]]), tensor([4.7868e-05])
prob: tensor([[1.0972e-06, 1.3324e-07]]), tensor([1.2304e-06])
prob: tensor([[7.9876e-08, 7.2277e-08]]), tensor([1.5215e-07])
prob: tensor([[6.6188e-08, 3.2096e-08]]), tensor([9.8284e-08])
prob: tensor([[1.4922e-07, 7.4672e-08]]), tensor([2.2389e-07])
prob: tensor([[2.0782e-07, 1.0903e-07]]), tensor([3.1685e-07])
prob: tensor([[1.2597e-07, 7.1883e-08]]), tensor([1.9785e-07])
prob: tensor([[8.4811e-08, 2.4370e-08]]), tensor([1.0918e-07])
prob: tensor([[3.0976e-08, 2.0622e-08]]), tensor([5.1598e-08])
prob: tensor([[2.1604e-08, 2.0281e-08]]), tensor([4.1884e-08])
prob: tensor([[7.9189e-08, 2.3159e-08]]), tensor([1.0235e-07])
prob: tensor([[2.9419e-08, 1.5496e-08]]), tensor([4.4915e-08])
prob: tensor([[6.8682e-08, 3.9354e-08]]), tensor([1.0804e-07])
prob: tensor([[3.9881e-08, 2.0365e-08]]), tensor([6.0246e-08])
prob: tensor([[2.4225e-08, 2.1114e-08]]), tensor([4.533

In [5]:
print(test_dataset[sample_idx]["sentence"], test_dataset[sample_idx]["input_speaker"])

Il améliore la lutte contre les incendies en renforçant les corps de sapeurs-pompiers. v2/en_speaker_1


In [6]:
from IPython.display import Audio as PlayAudio
PlayAudio(inputs["input_values"], rate=16_000)

In [7]:
PlayAudio(list(pre_trained_generated_speech), rate=16_000)

In [8]:
PlayAudio(list(fine_tuned_generated_speech), rate=16_000)

In [9]:
PlayAudio(list(fine_tuned_generated_speech), rate=16_000)

# Get Suno Embedding

In [10]:
import torch
from datasets import load_dataset
from transformers import BarkModel, BarkProcessor


DEVICE = "cpu"

test_dataset = load_dataset("preetam8/accent_coach_training_dataset", split="test")
sample_idx = 0

def initialize_model(small=True):
    model_name = "suno/bark-small" if small else "suno/bark"
    model = BarkModel.from_pretrained(model_name).to(DEVICE)
    processor = BarkProcessor.from_pretrained(model_name)
    return model, processor

suno_model, suno_processor = initialize_model()

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

Overwrite dataset info from restored data version if exists.
Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
Found cached dataset accent_coach_training_dataset (/Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee)
Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
Constructing Dataset for split test, from /Users/preetamamancharla/.cache/huggingface/datasets/preetam8___accent_coach_training_dataset/default/0.0.0/0743cc6de10b27f45ca1910f6871b7bea597e5ee
/Users/preetamamancharla/.pyenv/versions/3.11.5/envs/accent-coach-env/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor

In [11]:
sentence = test_dataset[sample_idx]["sentence"]
suno_inputs = suno_processor([sentence], voice_preset="v2/fr_speaker_7")
print(sentence)

Il améliore la lutte contre les incendies en renforçant les corps de sapeurs-pompiers.


In [12]:
signal = suno_model.generate(**suno_inputs).numpy()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [13]:
from IPython.display import Audio as PlayAudio
PlayAudio(signal, rate=24_000)

In [ ]:
suno_large_model, suno_large_processor = initialize_model(small=False)
suno_large_inputs = suno_large_processor([sentence], voice_preset="v2/fr_speaker_7")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/preetamamancharla/.pyenv/versions/3.11.5/envs/accent-coach-env/lib/python3.11/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [15]:
signal_from_large = suno_large_model.generate(**suno_large_inputs).numpy()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [16]:
PlayAudio(signal_from_large, rate=24_000)

In [26]:
from speechbrain.pretrained import SpeakerRecognition
verification = SpeakerRecognition.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_model")

# Extract embeddings
embeddings = verification.encode_batch(torch.Tensor(signal))
print(embeddings.shape)  # [Batch_size, Embedding_dim]


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch embedding_model.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch mean_var_norm_emb.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-xvect-voxceleb' if not cached
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder


torch.Size([1, 1, 512])


/Users/preetamamancharla/.pyenv/versions/3.11.5/envs/accent-coach-env/lib/python3.11/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental f

In [27]:
print(embeddings[0].shape)

torch.Size([1, 512])


In [19]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
x_vector = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
x_vector.shape

Loading the dataset from the Parquet export on Hugging Face.
INFO:datasets.load:Loading the dataset from the Parquet export on Hugging Face.
Overwrite dataset info from restored data version if exists.
INFO:datasets.builder:Overwrite dataset info from restored data version if exists.
Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/5c1297a9eb6c91714ea77c0d4ac5aca9b6a952e5
INFO:datasets.info:Loading Dataset info from /Users/preetamamancharla/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/5c1297a9eb6c91714ea77c0d4ac5aca9b6a952e5
Found cached dataset cmu-arctic-xvectors (/Users/preetamamancharla/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/5c1297a9eb6c91714ea77c0d4ac5aca9b6a952e5)
INFO:datasets.builder:Found cached dataset cmu-arctic-xvectors (/Users/preetamamancharla/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/5c1297a9eb6c91714ea77c

torch.Size([1, 512])

In [20]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import librosa
import torch

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

resampled_signal = librosa.resample(signal, orig_sr=24_000, target_sr=16_000)

inputs = processor(resampled_signal, sampling_rate=16_000, return_tensors="pt", padding=True)
with torch.no_grad():
    wav2vec = model(**inputs).last_hidden_state.mean(dim=1)  # Aggregate embeddings
print(wav2vec.shape)  # [Batch_size, Embedding_dim]


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([1, 768])


In [37]:
pre_trained_generated_speech = pre_trained_model.generate_speech(inputs["input_values"], speaker_embeddings=embeddings[0], vocoder=vocoder, maxlenratio=2.0)

prob: tensor([[8.5142e-10, 3.8066e-05]]), tensor([3.8066e-05])
prob: tensor([[5.8558e-10, 6.1313e-06]]), tensor([6.1319e-06])
prob: tensor([[2.4582e-10, 4.0277e-06]]), tensor([4.0280e-06])
prob: tensor([[1.9264e-10, 2.0389e-06]]), tensor([2.0390e-06])
prob: tensor([[5.0286e-10, 5.2732e-07]]), tensor([5.2782e-07])
prob: tensor([[4.6769e-10, 2.0403e-07]]), tensor([2.0450e-07])
prob: tensor([[3.7079e-10, 7.4430e-08]]), tensor([7.4800e-08])
prob: tensor([[4.7361e-10, 5.3804e-08]]), tensor([5.4278e-08])
prob: tensor([[4.4309e-10, 1.4819e-07]]), tensor([1.4864e-07])
prob: tensor([[3.9792e-10, 1.5682e-07]]), tensor([1.5721e-07])
prob: tensor([[8.7339e-10, 7.3181e-08]]), tensor([7.4054e-08])
prob: tensor([[1.7981e-09, 6.7483e-08]]), tensor([6.9281e-08])
prob: tensor([[6.2557e-10, 9.7454e-08]]), tensor([9.8079e-08])
prob: tensor([[1.6835e-10, 2.2237e-08]]), tensor([2.2406e-08])
prob: tensor([[5.9988e-10, 4.7455e-07]]), tensor([4.7515e-07])
prob: tensor([[1.2642e-09, 1.1051e-07]]), tensor([1.117

In [38]:
PlayAudio(pre_trained_generated_speech, rate=16_000)